In [8]:
import folium
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from shapely.geometry import box
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from shapely.geometry import Point
from pyproj import Proj, transform, CRS

In [2]:
import matplotlib.font_manager as fm

# 한국어 출력을 위한 폰트 설정
plt.rc('font', family='NanumGothic') 

# 마이너스 기호가 깨지는 것을 방지
plt.rcParams['axes.unicode_minus'] = False

In [5]:
df = pd.read_csv("data/서울시 소방서,안전센터,구조대 위치정보.csv", encoding='cp949')

In [6]:
df

,연번,서ㆍ센터ID,서ㆍ센터명,유형구분명,도서지역포함여부,상위서ㆍ센터ID,일련번호,X좌표,Y좌표
0,1,1116000,관악소방서,소방서,,1100000,43,195785.501520,541613.179889
1,2,1114103,영동119안전센터,안전센터/구조대,,1114000,44,203265.811458,546962.399786
2,3,1116401,관악119안전센터,안전센터/구조대,,1116000,45,195784.878412,541621.210326
3,4,1119106,시흥119안전센터,안전센터/구조대,,1119000,46,191196.721148,538802.571413
4,5,1115102,방배119안전센터,안전센터/구조대,,1115000,47,198957.096000,541787.001000
...,...,...,...,...,...,...,...,...,...
172,173,1125102,우이119안전센터,안전센터/구조대,,1125000,38,201461.941319,560141.801092
173,174,1125103,미아119안전센터,안전센터/구조대,,1125000,39,202500.723258,557825.979950
174,175,1111000,노원소방서,소방서,,1100000,40,206278.710106,559835.894278
175,176,1110000,도봉소방서,소방서,,1100000,41,203797.404500,562711.422384


In [9]:
proj_korea = Proj(init='epsg:5186')
# WGS84
proj_wgs84 = Proj(init='epsg:4326')

C:\Users\1qlqj\Desktop\multicamp_semi\multicamp_semi\venv\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\1qlqj\Desktop\multicamp_semi\multicamp_semi\venv\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [10]:
# X좌표와 Y좌표를 위도와 경도로 변환하는 함수
def convert_coords(x, y):
    lon, lat = transform(proj_korea, proj_wgs84, x, y)
    return lat, lon

In [11]:
# 모든 좌표 변환
df[['위도', '경도']] = df.apply(lambda row: convert_coords(row['X좌표'], row['Y좌표']), axis=1, result_type="expand")

C:\Users\1qlqj\AppData\Local\Temp\ipykernel_40368\3974450290.py:3: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lon, lat = transform(proj_korea, proj_wgs84, x, y)


In [12]:
df.head()

,연번,서ㆍ센터ID,서ㆍ센터명,유형구분명,도서지역포함여부,상위서ㆍ센터ID,일련번호,X좌표,Y좌표,위도,경도
0,1,1116000,관악소방서,소방서,,1100000,43,195785.501520,541613.179889,37.473943,126.952355
1,2,1114103,영동119안전센터,안전센터/구조대,,1114000,44,203265.811458,546962.399786,37.522143,127.036944
2,3,1116401,관악119안전센터,안전센터/구조대,,1116000,45,195784.878412,541621.210326,37.474015,126.952348
3,4,1119106,시흥119안전센터,안전센터/구조대,,1119000,46,191196.721148,538802.571413,37.448587,126.900512
4,5,1115102,방배119안전센터,안전센터/구조대,,1115000,47,198957.096000,541787.001000,37.475518,126.988210


In [13]:
df.columns

Index(['연번', '서ㆍ센터ID', '서ㆍ센터명', '유형구분명', '도서지역포함여부', '상위서ㆍ센터ID', '일련번호', 'X좌표',
       'Y좌표', '위도', '경도'],
      dtype='object')

In [14]:
df = df.drop(columns=['연번', '서ㆍ센터ID', '도서지역포함여부', '상위서ㆍ센터ID', '일련번호', 'X좌표',
       'Y좌표'])

In [31]:
df['유형구분명'].value_counts()

유형구분명
안전센터        119
구조대          32
소방서          24
안전센터/구조대      2
Name: count, dtype: int64

In [25]:
df.head()

,서ㆍ센터명,유형구분명,위도,경도
0,관악소방서,소방서,37.473943,126.952355
1,영동119안전센터,안전센터/구조대,37.522143,127.036944
2,관악119안전센터,안전센터/구조대,37.474015,126.952348
3,시흥119안전센터,안전센터/구조대,37.448587,126.900512
4,방배119안전센터,안전센터/구조대,37.475518,126.988210


In [33]:
df.loc[df['서ㆍ센터명'].str.contains('안전센터'), '유형구분명'] = '안전센터'
df.loc[df['서ㆍ센터명'].str.contains('구조대'), '유형구분명'] = '구조대'
df.loc[df['서ㆍ센터명'].str.contains('소방항공대'), '유형구분명'] = '소방항공대'
df.loc[df['서ㆍ센터명'].str.contains('본부현장대응단'), '유형구분명'] = '특수대응단'

In [34]:
df.head()

,서ㆍ센터명,유형구분명,위도,경도
0,관악소방서,소방서,37.473943,126.952355
1,영동119안전센터,안전센터,37.522143,127.036944
2,관악119안전센터,안전센터,37.474015,126.952348
3,시흥119안전센터,안전센터,37.448587,126.900512
4,방배119안전센터,안전센터,37.475518,126.988210


In [36]:
# 서울 중심부에 folium 지도 생성
m = folium.Map(location=[37.5642135, 127.0016985], zoom_start=11)

# 유형에 따른 마커 색상 지정
colors = {
    '소방서': 'red',
    '안전센터': 'blue',
    '구조대': 'orange',
    '소방항공대':'lightblue',
    '특수대응단':'purple'
    
}

# 각 행에 대해 마커 추가
for index, row in df.iterrows():
    popup_content = f"<b>서센터명:</b> {row['서ㆍ센터명']}<br><b>유형구분명:</b> {row['유형구분명']}"
    folium.Marker(
        location=[row['위도'], row['경도']],
        popup=folium.Popup(popup_content, max_width=300),
        icon=folium.Icon(color=colors[row['유형구분명']])
    ).add_to(m)

# 지도 표시
m

In [37]:
df.to_csv("data/서울시_소방서_안전세터_구조대.csv")